# 🎬 Baseline Workflow

This notebook implements the baseline workflow. Each scene is generated independently using only its description from the episode outline and a rolling window of up to three previously generated scripts. These prior scenes are passed to the language model as raw text, without summaries, planning signals, or agent feedback. The LLM operates purely autoregressively, with no structured memory or mechanism for long-term narrative tracking.


## Mounting Drive and Appending System

In [ ]:
# Import necessary modules
from google.colab import drive
import sys

# Mount Google Drive to access files stored in your Google Drive
drive.mount('/content/drive')

# NOTE: Update the paths below to match the location of your project files in Google Drive.
# Replace with your own directory if different.

# Add the main 'utils' directory to Python's module search path
# This allows you to import custom utility modules from this folder
sys.path.append("/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils")

# Add the 'agents' subdirectory to the module search path
# Useful if you have agent-specific Python scripts you want to import
sys.path.append("/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils/agents")

MessageError: Error: credential propagation was unsuccessful

## Importing Libraries and Packages

In [ ]:
# Import text utility functions for formatting and extracting key components
from text_utils import (
    display_markdown_output,
    extract_scene,
)

# Import scene writing functions
from screen_writing import (
    generate_scene_1_script,
    generate_scene_baseline
)

In [ ]:
from openai import OpenAI
from google.colab import userdata
import os
import re
from IPython.display import Markdown, display

### 🔑 Initializing the OpenAI API Client

In [ ]:
api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

### Reading Title and Episode Outline


In [ ]:
# Define file paths
title_path = "/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/pilot episode data/pilot episode 5/sitcom_title_5.txt"
outline_path = "/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/pilot episode data/pilot episode 5/episode_outline_5.txt"

# Function to read text files
def read_text_file(path):
    with open(path, "r") as f:
        return f.read()

# Load data
sitcom_title = read_text_file(title_path)
episode_outline = read_text_file(outline_path)

# Print previews
print("Sitcom Title:")
print(sitcom_title)
print("\n" + "="*80 + "\n")
print("Full Episode Outline:")
print(episode_outline)

Sitcom Title:
Unlocked Potential


Full Episode Outline:
Episode Concept: In the pilot episode, Frankie is released from prison and discovers he has inherited his uncle's locksmith shop; meanwhile, he meets his quirky parole officer, Len, and reconnects with his estranged daughter, Ellie.

Scene 1: "Fresh Start" Frankie is released from prison, with a mix of joy and nervousness as he starts his new life.

Scene 2: "Unexpected Inheritance" Frankie discovers he has inherited his late uncle's locksmith shop, a surprising and overwhelming revelation.

Scene 3: "First Encounter" Frankie meets Len, his oddball parole officer, for the first time, leading to comedic misunderstandings.

Scene 4: "Home Sweet Home?" Frankie moves into the small apartment above the locksmith shop, finding it filled with his uncle's strange collection of locks and keys.

Scene 5: "The Shop Downstairs" Frankie explores the locksmith shop, overwhelmed by the complexity of the trade he's expected to take up.

Scene 6:

## 🎬 Scene 1 Generation

Scene generation is performed using the `generate_scene_1_script` function, which is also used in the ReAct workflow. It takes in the sitcom title and scene description as inputs but is applied here without any ReAct-style reasoning or agent feedback.


In [ ]:
# Extract the description for Scene 1 from the episode outline
scene_1_desc = extract_scene(episode_outline, 1)
print(scene_1_desc)

Scene 1: "Fresh Start" Frankie is released from prison, with a mix of joy and nervousness as he starts his new life.


In [ ]:
# Generate the full script for Scene 1 based on the scene description
scene_1_script = generate_scene_1_script(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_1_desc,
    scene_index=0,
    rag_context=None,
)

print(scene_1_script)

# Scene 1

INT. PRISON GATES - DAY

FRANKIE, early 50s, a bit worn but with a twinkle in his eye, steps through the prison gates, suitcase in hand.

FRANKIE
(squinting against the sunlight)
Well, Frankie, welcome to the rest of your life.

[LAUGH TRACK]

A beat-up taxi pulls up. AL, the taxi driver, 60s, gruff but friendly, leans out the window.

AL
You Frankie?

FRANKIE
(smiles)
Depends. You the Feds?

[LAUGH TRACK]

AL
(chuckles)
Nope, just your chariot to freedom.

FRANKIE
(sincere)
Thank God. Thought I was going back for a moment there.

[LAUGH TRACK]

Frankie gets in the taxi. He looks back at the prison, a bit melancholy.

FRANKIE
(to himself)
Goodbye, old life.

AL
(turning around)
You say something?

FRANKIE
(startled)
No, no. Just talking to myself. Comes with the territory.

[LAUGH TRACK]

Al starts driving. Frankie looks out the window, soaking in the view.

AL
(concerned)
You okay back there?

FRANKIE
(smiles)
Yeah, Al. I'm just getting used to seeing things without bars.



## 🎬 Scene 2 Generation

Scene 2 is generated using the full script of Scene 1 and the description of Scene 2. This pattern continues throughout the baseline workflow:  
- Scene 3 is generated using the scripts from Scenes 1 and 2, along with its own description.  
- Scene 4 and all subsequent scenes are generated using a rolling window of the previous three full scripts plus the current scene’s description.

This approach ensures that each scene has access to recent narrative context, but only in the form of raw text—without structured metadata, agent guidance, or planning signals.

In [ ]:
# Extract the description for Scene 2 from the episode outline
scene_2_desc = extract_scene(episode_outline, 2)
print(scene_2_desc)

Scene 2: "Unexpected Inheritance" Frankie discovers he has inherited his late uncle's locksmith shop, a surprising and overwhelming revelation.


In [ ]:
# Generate the full script for Scene 2 based on its description and the completed Scene 1 script
scene_2_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_2_desc,
    previous_scenes=[scene_1_script]
)

print(scene_2_script)

INT. EARL'S LOCKSMITH SHOP - DAY

Al pulls up outside a rundown locksmith shop. A sign hangs precariously above the door: "EARL'S LOCKS & KEYS."

AL
(reading the sign)
Earl's Locks & Keys? This your place?

FRANKIE
(confused)
I don't think so...

Al points to a letter lying on the passenger seat. Frankie picks it up and reads it. His eyes widen.

FRANKIE
(astonished)
Holy... This is my uncle's shop. He left it to me.

AL
(surprised)
Well, ain't that something?

[LAUGH TRACK]

Frankie gets out of the taxi, staring at the shop. He approaches the door, hesitates, then slowly opens it.

INT. EARL'S LOCKSMITH SHOP - CONTINUOUS

Frankie steps inside, taking in the dust-covered counters, jumbled keys, and worn-out lock displays.

FRANKIE
(mutters to himself)
Well, Frankie, you wanted a fresh start.

[LAUGH TRACK]

Suddenly, a CAT jumps from a high shelf, startling Frankie.

FRANKIE
(yelps)
Jesus!

[LAUGH TRACK]

He catches his breath, then laughs, shaking his head.

FRANKIE
(to the cat)
Well,

## 🎬 Scene 3 Generation

In [ ]:
# Extract the description for Scene 3 from the episode outline
scene_3_desc = extract_scene(episode_outline, 3)
display(Markdown(scene_3_desc))

Scene 3: "First Encounter" Frankie meets Len, his oddball parole officer, for the first time, leading to comedic misunderstandings.

In [ ]:
# Generate the full script for Scene 3 using its description and the completed scripts for Scenes 1 and 2 as context
scene_3_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_3_desc,
    previous_scenes=[scene_1_script, scene_2_script]
)

print(scene_3_script)

INT. EARL'S LOCKSMITH SHOP - DAY

Frankie is still behind the counter, sorting through the mess. Suddenly, the door jingles and LEN, a quirky man in his 40s with a clipboard, steps inside.

LEN
(looking at clipboard)
Frankie?

FRANKIE
(turns, startled)
Jesus! You people really need to stop sneaking up on me!

[LAUGH TRACK]

LEN
(laughs)
Sorry about that. I'm Len, your parole officer.

FRANKIE
(relieved)
Oh, thank God. Thought you were the taxman.

[LAUGH TRACK]

Len chuckles and extends his hand to Frankie.

LEN
(grinning)
Nah, just here to make sure you're not up to any old tricks.

Frankie shakes Len's hand, a bit wary.

FRANKIE
(sarcastic)
Well, if my old tricks involve dusting and sorting keys, then I'm guilty as charged.

[LAUGH TRACK]

Len laughs and glances around the shop.

LEN
(amused)
Your uncle left you quite the place.

FRANKIE
(smiling)
Yeah, he always had a thing for locks. Said they were like people. Every one unique and full of potential.

Len looks at Frankie, intrigue

## 🎬 Scene 4 Generation

In [ ]:
# Extract the description for Scene 4 from the episode outline
scene_4_desc = extract_scene(episode_outline, 4)
display(Markdown(scene_4_desc))

Scene 4: "Home Sweet Home?" Frankie moves into the small apartment above the locksmith shop, finding it filled with his uncle's strange collection of locks and keys.

In [ ]:
# Generate the full script for Scene 4 using its description and the completed scripts for Scenes 1 to 3 as context
scene_4_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_4_desc,
    previous_scenes=[scene_1_script, scene_2_script, scene_3_script]
)

print(scene_4_script)

INT. EARL'S LOCKSMITH SHOP - UPSTAIRS APARTMENT - DAY

Frankie climbs up a narrow staircase, carrying a suitcase. He opens the door to the apartment, revealing an equally cluttered space filled with lock-related paraphernalia.

FRANKIE
(looking around)
Well, at least the decor's consistent.

[LAUGH TRACK]

Frankie steps inside, accidentally knocking over a stack of lock-picking books.

FRANKIE
(startled)
Whoa!

[LAUGH TRACK]

He picks up one of the books and flips through it.

FRANKIE
(reading aloud)
"Unlocking the Mysteries of Locks: A Comprehensive Guide."

He looks at the book, then at the room filled with locks and keys.

FRANKIE
(mocking)
Oh great, bedtime reading.

[LAUGH TRACK]

Suddenly, the CAT from the shop jumps onto a nearby table, causing a cascade of keys to fall onto the floor.

FRANKIE
(yelps)
Not you again!

[LAUGH TRACK]

He sighs, looking around the room. His gaze falls on a beautiful antique lock displayed on a shelf.

FRANKIE
(picks it up)
Well, aren't you a beauty

## 🎬 Scene 5 Generation

In [ ]:
# Extract the description for Scene 5 from the episode outline
scene_5_desc = extract_scene(episode_outline, 5)
print(scene_5_desc)

Scene 5: "The Shop Downstairs" Frankie explores the locksmith shop, overwhelmed by the complexity of the trade he's expected to take up.


In [ ]:
# Generate the full script for Scene 5 using its description and the completed scripts for Scenes 2 to 4 as context
scene_5_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_5_desc,
    previous_scenes=[scene_2_script, scene_3_script, scene_4_script]
)

print(scene_5_script)

INT. EARL'S LOCKSMITH SHOP - DAY

Frankie is standing behind the counter, examining a lock with a complex design. He looks frustrated as he tries to pick it. The CAT watches from the counter, swishing its tail.

FRANKIE
(grumbling)
Unlocking potential my foot...

[LAUGH TRACK]

Suddenly, the lock clicks open. Frankie looks surprised, then triumphant.

FRANKIE
(pumps fist)
Yes! Eat that, lock!

[LAUGH TRACK]

The cat meows, unimpressed. Frankie glares at it.

FRANKIE
(defensive)
Hey, I don't see you picking any locks.

The cat yawns and stretches, ignoring him. Frankie grins and ruffles its fur.

FRANKIE
(smiling)
That's what I thought.

Suddenly, the door jingles and a WOMAN, mid 30s, stylish and flustered, enters.

WOMAN
(anxious)
Oh thank God, you're open. I need a key duplicated. Can you do it right now?

Frankie looks taken aback, then nods, trying to appear confident.

FRANKIE
(confident)
Of course, ma'am. No job too big or small for Frankie's Locksmith services.

The woman hands 

## 🎬 Scene 6 Generation

In [ ]:
# Extract the description for Scene 6 from the episode outline
scene_6_desc = extract_scene(episode_outline, 6)
print(scene_6_desc)

Scene 6: "Len's Magic Obsession" Len visits Frankie at the shop and reveals his obsession with magic tricks, leading to an awkwardly funny magic trick demonstration.


In [ ]:
# Generate the full script for Scene 6 using its description and the completed scripts for Scenes 3 to 5 as context
scene_6_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_6_desc,
    previous_scenes=[scene_3_script, scene_4_script, scene_5_script]
)

print(scene_6_script)

INT. EARL'S LOCKSMITH SHOP - DAY

Frankie is behind the counter, examining another intricate lock. The CAT is batting at a key hanging from a string. The door jingles and LEN enters, a wide grin on his face and a deck of cards in his hand.

LEN
(beaming)
Frankie, my man!

FRANKIE
(startled)
Len! You gotta stop doing that!

[LAUGH TRACK]

LEN
(laughs)
Sorry, couldn't resist. How's the unlocking going?

FRANKIE
(moans)
It's more like I'm unlocking a headache.

[LAUGH TRACK]

Len chuckles, then glances at the deck of cards in his hand.

LEN
(slyly)
Well, maybe you need a break. Fancy a magic trick?

Frankie raises an eyebrow, intrigued.

FRANKIE
(curious)
Magic? You?

LEN
(smiling)
Oh yeah. I'm a regular Houdini.

Frankie chuckles, leaning back on the counter.

FRANKIE
(amused)
Alright, Len. Let's see what you got.

Len grins, shuffling the deck of cards. He performs a few simple tricks, much to Frankie's amusement. Then, he attempts a more complex trick, but the cards slip from his hands

## 🎬 Scene 7 Generation

In [ ]:
# Extract the description for Scene 7 from the episode outline
scene_7_desc = extract_scene(episode_outline, 7)
print(scene_7_desc)

Scene 7: "Surprise Visit" Frankie's estranged daughter, Ellie, unexpectedly shows up at the locksmith shop, adding tension and unexpected humor.


In [ ]:
# Generate the full script for Scene 7 using its description and the completed scripts for Scenes 4 to 6 as context
scene_7_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_7_desc,
    previous_scenes=[scene_4_script, scene_5_script, scene_6_script]
)

print(scene_7_script)

INT. EARL'S LOCKSMITH SHOP - DAY

Frankie is behind the counter, picking at a particularly tricky lock. The CAT is perched on the counter, watching a fly buzz around. The door jingles and ELLIE, late 20s, free-spirited and unconventional, enters.

FRANKIE
(surprised)
Ellie?

ELLIE
(smiling)
Hey, Dad.

Frankie looks shocked, then recovers.

FRANKIE
(stammering)
What...how...when did you get into town?

ELLIE
(shrugs)
Just now. Thought I'd surprise you.

FRANKIE
(chuckles)
Well, you definitely did that.

[LAUGH TRACK]

Ellie looks around the shop, intrigued.

ELLIE
(looking at locks)
So, this is what you've been doing. Playing with locks all day.

FRANKIE
(defensive)
It's not playing, it's...it's serious business.

Ellie picks up a lock, studying it.

ELLIE
(slyly)
Oh, I can see that. The look of intense concentration, the furrowed brow...

Frankie frowns, looking at her.

FRANKIE
(skeptical)
Are you making fun of me?

Ellie grins, holding up her hands in surrender.

ELLIE
(laughs)
Would

## 🎬 Scene 8 Generation

In [ ]:
# Extract the description for Scene 8 from the episode outline
scene_8_desc = extract_scene(episode_outline, 8)
print(scene_8_desc)

Scene 8: "Father-Daughter Reunion" Frankie and Ellie have an awkward and emotional reunion, struggling to reconnect after years of estrangement.


In [ ]:
# Generate the full script for Scene 8 using its description and the completed scripts for Scenes 5 to 7 as context
scene_8_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_8_desc,
    previous_scenes=[scene_5_script, scene_6_script, scene_7_script]
)

print(scene_8_script)

INT. EARL'S LOCKSMITH SHOP - DAY

Frankie is still behind the counter, Ellie is examining a lock. The CAT is playing with a feather.

FRANKIE
(amused)
You sure you want to learn this?

ELLIE
(nods)
Yeah, Dad. It's about time I learned the family business.

FRANKIE
(surprised)
Family business? Ellie, it's just me and a bunch of locks here.

[LAUGH TRACK]

Ellie grins, picking up a lock.

ELLIE
(teasing)
And don't forget the cat.

Frankie chuckles, shaking his head.

FRANKIE
(laughs)
Oh, right. The cat.

[LAUGH TRACK]

Frankie hands her a lock and a set of picks. Ellie looks at them, then at the lock, confused.

FRANKIE
(encouraging)
Just feel it out, Ellie. It's all about the touch.

Ellie looks skeptical, then tries to pick the lock. She fumbles, dropping the picks. 

ELLIE
(frustrated)
This is impossible!

Frankie chuckles, shaking his head.

FRANKIE
(teasing)
You've barely started, Ellie. It's not rocket science.

[LAUGH TRACK]

Ellie huffs, picking up the picks again. She tries agai

## 🎬 Scene 9 Generation

In [ ]:
# Extract the description for Scene 9 from the episode outline
scene_9_desc = extract_scene(episode_outline, 9)
print(scene_9_desc)

Scene 9: "Ellie Moves In" Ellie, having nowhere else to go, moves in with Frankie, adding to the chaos in his life.


In [ ]:
# Generate the full script for Scene 9 using its description and the completed scripts for Scenes 6 to 8 as context
scene_9_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_9_desc,
    previous_scenes=[scene_6_script, scene_7_script, scene_8_script]
)

print(scene_9_script)

INT. EARL'S LOCKSMITH SHOP - DAY

Frankie and Ellie are still at the counter, Ellie struggling with the lock. The CAT is watching from a distance, its tail twitching.

ELLIE
(frustrated)
This is harder than it looks!

FRANKIE
(smirks)
Welcome to my world.

[LAUGH TRACK]

Ellie huffs, setting down the lock and picks.

ELLIE
(defeated)
Maybe I should just stick to things I'm good at.

FRANKIE
(teasing)
Like moving around and making my life chaotic?

[LAUGH TRACK]

Ellie grins, shrugging.

ELLIE
(laughs)
Well, I am pretty good at that.

Frankie chuckles, nodding.

FRANKIE
(smiling)
That you are, kiddo.

Ellie looks around, then back at Frankie.

ELLIE
(serious)
Dad, I need a place to stay.

Frankie looks taken aback, then recovers.

FRANKIE
(surprised)
You want to stay here?

Ellie nods, looking hopeful.

ELLIE
(nods)
If that's okay with you.

Frankie looks thoughtful, then smiles, nodding.

FRANKIE
(smiling)
Of course, Ellie. You're always welcome here.

Ellie grins, relief washing over 

## 🎬 Scene 10 Generation

In [ ]:
# Extract the description for Scene 10 from the episode outline
scene_10_desc = extract_scene(episode_outline, 10)
print(scene_10_desc)

Scene 10: "Locksmith Lessons" Frankie attempts to learn the locksmith trade, resulting in hilariously disastrous attempts at fixing locks.


In [ ]:
# Generate the full script for Scene 10 using its description and the completed scripts for Scenes 7 to 9 as context
scene_10_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_10_desc,
    previous_scenes=[scene_7_script, scene_8_script, scene_9_script]
)

print(scene_10_script)

INT. EARL'S LOCKSMITH SHOP - DAY

Frankie and Ellie are still at the counter, Ellie fumbling with the lock. The CAT is batting a feather on the floor.

ELLIE
(frustrated)
This is impossible!

FRANKIE
(teasing)
Welcome to my world, Ellie.

[LAUGH TRACK]

Ellie sighs, setting the lock and picks down.

ELLIE
(defeated)
Maybe locksmithing isn't my calling.

Frankie smiles, shaking his head.

FRANKIE
(amused)
It's not for everyone. But you're not off the hook. 

Ellie raises an eyebrow, intrigued.

ELLIE
(curious)
Oh? What's next then?

Frankie grins, revealing a tiny, delicate lock.

FRANKIE
(smiling)
Lock picking for beginners.

Ellie sighs dramatically, picking up the lock and picks.

ELLIE
(determined)
Alright, bring it on.

[LAUGH TRACK]

Frankie chuckles, watching as Ellie starts to fumble with the new lock.

FRANKIE
(teasing)
Patience, Ellie. Patience.

Ellie huffs, but continues with her task. Suddenly, there's a CLICK. Ellie looks up, surprised.

ELLIE
(surprised)
Did I...?

Franki

## 🎬 Scene 11 Generation

In [ ]:
# Extract the description for Scene 11 from the episode outline
scene_11_desc = extract_scene(episode_outline, 11)
print(scene_11_desc)

Scene 11: "Philosophy with Len" Len uses his philosophical approach to life to encourage Frankie, leading to humorous and deep conversations.


In [ ]:
# Generate the full script for Scene 11 using its description and the completed scripts for Scenes 8 to 10 as context
scene_11_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_11_desc,
    previous_scenes=[scene_8_script, scene_9_script, scene_10_script]
)

print(scene_11_script)

INT. EARL'S LOCKSMITH SHOP - DAY

Frankie is at the counter, working on a lock. Ellie emerges from the back, stretching. Len, a regular customer, enters the shop, his friendly smile on display.

LEN
(cheerfully)
Hello, Frankie! And Ellie, back again I see.

ELLIE
(smiling)
Hi, Len. Yes, trying to unlock my hidden talents.

[LAUGH TRACK]

Len chuckles, approaching the counter.

LEN
(grinning)
I see what you did there. 

Frankie smiles, shaking his head.

FRANKIE
(amused)
She's been here all day, Len. I think the lock picking's gone to her head.

[LAUGH TRACK]

Len laughs, looking at Ellie.

LEN
(philosophical)
Well, Ellie, just remember - a lock is like life. You can't force it. You have to feel it out, find the right combination.

Ellie looks at Len, impressed.

ELLIE
(wide-eyed)
Wow, Len. That's deep.

Len chuckles, shrugging.

LEN
(smiling)
Just something I picked up along the way.

Frankie rolls his eyes, grinning.

FRANKIE
(teasing)
Don't encourage him, Ellie. He'll start talking a

## 🎬 Scene 12 Generation

In [ ]:
# Extract the description for Scene 12 from the episode outline
scene_12_desc = extract_scene(episode_outline, 12)
print(scene_12_desc)

Scene 12: "First Customer" Frankie and Len deal with their first customer, a situation filled with confusion and blunders.


In [ ]:
# Generate the full script for Scene 12 using its description and the completed scripts for Scenes 9 to 11 as context
scene_12_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_12_desc,
    previous_scenes=[scene_9_script, scene_10_script, scene_11_script]
)

print(scene_12_script)

INT. EARL'S LOCKSMITH SHOP - DAY

Frankie is at the counter, working on a lock. Ellie is sorting through a box of keys. The CAT is asleep on the counter. The DOORBELL RINGS, and a NERVOUS WOMAN enters, looking around.

NERVOUS WOMAN
(awkwardly)
Um, hi. I need a key made?

Frankie glances at Ellie, who straightens up, eager.

FRANKIE
(slyly)
Ellie, why don’t you handle this one?

ELLIE
(surprised)
Really? You trust me with a customer?

FRANKIE
(teasing)
I didn’t say that. I just said handle it.

[LAUGH TRACK]

Ellie rolls her eyes, but steps forward, smiling at the Nervous Woman.

ELLIE
(smiling)
Of course. What kind of key do you need?

Nervous Woman hands Ellie a small, intricate key. Ellie takes it, her eyes wide.

ELLIE
(nervously)
Oh, this is... this is a nice key.

[LAUGH TRACK]

Nervous Woman raises an eyebrow.

NERVOUS WOMAN
(confused)
Yes... Can you duplicate it?

Ellie glances at Frankie, who gives her a thumbs up. Ellie nods, turning back to the Nervous Woman.

ELLIE
(confide

## 🎬 Scene 13 Generation

In [ ]:
# Extract the description for Scene 13 from the episode outline
scene_13_desc = extract_scene(episode_outline, 13)
print(scene_13_desc)

Scene 13: "Ellie's Street Smarts" Ellie uses her street-smart skills to help Frankie and Len deal with a difficult customer, showing her potential.


In [ ]:
# Generate the full script for Scene 13 using its description and the completed scripts for Scenes 10 to 12 as context
scene_13_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_13_desc,
    previous_scenes=[scene_10_script, scene_11_script, scene_12_script]
)

print(scene_13_script)

INT. EARL'S LOCKSMITH SHOP - DAY

Frankie is at the counter, working on a lock. Ellie is cleaning up the spilled keys. Len is browsing locks. The CAT is back to sleep on the counter. The DOORBELL RINGS, and a GRUFF MAN enters, looking annoyed.

GRUFF MAN
(gruffly)
I need a lock opened. Now!

Frankie looks surprised, then grins at Ellie.

FRANKIE
(slyly)
Ellie, want to give it a shot?

ELLIE
(surprised)
Wait, you mean I should pick a lock? In front of a customer?

Frankie shrugs, grinning.

FRANKIE
(teasing)
Why not? You've unlocked your potential, haven't you?

[LAUGH TRACK]

Ellie rolls her eyes, but steps forward, smiling nervously at the Gruff Man.

ELLIE
(nervously)
Sure, I can do that. 

She takes the lock from the Gruff Man and begins to work on it, her hands shaking. Frankie and Len watch, amused.

LEN
(laughs)
This should be interesting.

Frankie nods, grinning.

FRANKIE
(amused)
Oh, it will be.

[LAUGH TRACK]

Ellie struggles with the lock, her face turning red. The Gruff Man 

## 🎬 Scene 14 Generation

In [ ]:
# Extract the description for Scene 14 from the episode outline
scene_14_desc = extract_scene(episode_outline, 14)
print(scene_14_desc)

Scene 14: "Dinner Time Disaster" The trio tries to have a peaceful dinner together, which inevitably turns into a comedic disaster.


In [ ]:
# Generate the full script for Scene 14 using its description and the completed scripts for Scenes 11 to 13 as context
scene_14_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_14_desc,
    previous_scenes=[scene_11_script, scene_12_script, scene_13_script]
)

print(scene_14_script)

INT. EARL'S LOCKSMITH SHOP - DAY

Ellie is examining the complex lock, a look of determination on her face. Frankie is grinning at her. Len is chuckling in the background. The CAT is asleep on the counter. The DOORBELL RINGS, and a DELIVERY GUY enters with a huge box.

DELIVERY GUY
(struggling)
Delivery for... Frankie?

Frankie looks surprised, then grins at Ellie.

FRANKIE
(slyly)
Ellie, why don’t you sign for this?

ELLIE
(confused)
Me? But it's not my delivery.

Frankie shrugs, grinning.

FRANKIE
(teasing)
Consider it another part of your locksmith training.

[LAUGH TRACK]

Ellie rolls her eyes, but steps forward, signing for the delivery. The Delivery Guy leaves, and Ellie turns to Frankie, looking at the box.

ELLIE
(curious)
So, what's in the box?

Frankie grins, opening the box to reveal a large, elaborate LOCK.

FRANKIE
(amused)
Just a little something to keep you on your toes.

[LAUGH TRACK]

Ellie groans, but smiles, picking up the lock.

ELLIE
(teasing)
Is this your way of s

## 🎬 Scene 15 Generation

In [ ]:
# Extract the description for Scene 15 from the episode outline
scene_15_desc = extract_scene(episode_outline, 15)
print(scene_15_desc)

Scene 15: "Lock Metaphors" Len uses a lock and key as a metaphor for Frankie's situation, leading to laughter and self-realization.


In [ ]:
# Generate the full script for Scene 15 using its description and the completed scripts for Scenes 12 to 14 as context
scene_15_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_15_desc,
    previous_scenes=[scene_12_script, scene_13_script, scene_14_script]
)

print(scene_15_script)

INT. FRANKIE'S APARTMENT - NIGHT

They're still sitting around the small table. The pizza boxes are empty, and Ellie's complex lock is now disassembled on the table. The CAT is sleeping in Len's lap. Frankie is wiping pizza grease off his hands and smirking at Ellie.

FRANKIE
(smiling)
So, how's that lock coming along, kiddo?

Ellie looks at the disassembled lock parts scattered on the table, then at Frankie.

ELLIE
(defiant)
Better than your pizza cutting skills.

[LAUGH TRACK]

Frankie chuckles, picking up a tiny lock piece with a napkin.

FRANKIE
(teasing)
This little piece is the key to your success, you know.

Ellie raises an eyebrow, looking at Frankie.

ELLIE
(skeptical)
Really? How so?

Frankie smirks, holding up the tiny piece.

FRANKIE
(slyly)
Well, you see, this piece is like... your ability to handle chaos. Without it, the whole lock falls apart.

Len laughs, petting the Cat.

LEN
(laughs)
And without chaos, this place would be pretty boring.

[LAUGH TRACK]

Ellie smiles, p

## 🎬 Scene 16 Generation

In [ ]:
# Extract the description for Scene 16 from the episode outline
scene_16_desc = extract_scene(episode_outline, 16)
print(scene_16_desc)

Scene 16: "Late Night Bonding" Frankie and Ellie share a heartfelt conversation late at night, taking steps towards mending their relationship.


In [ ]:
# Generate the full script for Scene 16 using its description and the completed scripts for Scenes 13 to 15 as context
scene_16_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_16_desc,
    previous_scenes=[scene_13_script, scene_14_script, scene_15_script]
)

print(scene_16_script)

INT. FRANKIE'S APARTMENT - LATE NIGHT

The table is now clear, except for the reassembled lock and a few crumbs. Len has left, and the CAT is curled up asleep in the corner. Frankie and Ellie are sitting across from each other, looking at the lock.

FRANKIE
(smiling)
You know, you did a pretty good job with that.

ELLIE
(surprised)
Really?

Frankie nods, picking up the lock and examining it.

FRANKIE
(amused)
Yeah, I mean, you only broke it twice. That's a new record.

[LAUGH TRACK]

Ellie rolls her eyes, but smiles.

ELLIE
(teasing)
Well, I'm glad I could entertain you.

Frankie chuckles, setting the lock back down.

FRANKIE
(smiling)
You know, Ellie, it's not just about the entertainment.

Ellie looks at Frankie, curious.

ELLIE
(confused)
Then what is it about?

Frankie looks thoughtful, choosing his words carefully.

FRANKIE
(serious)
It's about unlocking potential. Yours, mine, even that grumpy cat's.

Ellie laughs, looking at the sleeping Cat.

ELLIE
(teasing)
Well, his potential

## 🎬 Scene 17 Generation

In [ ]:
# Extract the description for Scene 17 from the episode outline
scene_17_desc = extract_scene(episode_outline, 17)
print(scene_17_desc)

Scene 17: "Shop Improvements" The trio work together to improve the locksmith shop, leading to comedic renovations and bonding moments.


In [ ]:
# Generate the full script for Scene 17 using its description and the completed scripts for Scenes 14 to 16 as context
scene_17_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_17_desc,
    previous_scenes=[scene_14_script, scene_15_script, scene_16_script]
)

print(scene_17_script)

INT. EARL'S LOCKSMITH SHOP - MORNING

The shop is cluttered with tools, locks, and keys. Frankie is at the counter, sipping coffee. Ellie enters the shop, looking refreshed but slightly nervous. Len is already there, organizing keys. The CAT is asleep on a pile of locks.

ELLIE
(smiling)
Good morning, Frankie. Len.

FRANKIE
(teasing)
Morning, Ellie. Ready for your safe-cracking session?

LEN
(laughs)
More like safe-cracking chaos.

[LAUGH TRACK]

Ellie rolls her eyes, but grins.

ELLIE
(teasing)
Hey, I handled that lock last night, didn't I?

Frankie chuckles, raising his coffee cup in a toast.

FRANKIE
(smiling)
You did, kiddo. You did.

Ellie looks around the shop, an idea forming.

ELLIE
(curious)
You know, this place could use a little sprucing up.

Frankie raises an eyebrow, looking around.

FRANKIE
(amused)
Oh, really? And what do you suggest?

Ellie grins, looking at Len.

ELLIE
(slyly)
How about a little shop renovation?

Len chuckles, shaking his head.

LEN
(laughs)
Now that's

## 🎬 Scene 18 Generation

In [ ]:
# Extract the description for Scene 18 from the episode outline
scene_18_desc = extract_scene(episode_outline, 18)
print(scene_18_desc)

Scene 18: "Second Chance" Frankie reflects on his second chance at life, leading to a heartfelt monologue about redemption and hope.


In [ ]:
# Generate the full script for Scene 18 using its description and the completed scripts for Scenes 15 to 17 as context
scene_18_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_18_desc,
    previous_scenes=[scene_15_script, scene_16_script, scene_17_script]
)

print(scene_18_script)

INT. EARL'S LOCKSMITH SHOP - LATE AFTERNOON

The shop is half-cleaned, half-chaotic. Ellie is on a ladder, dusting a high shelf. Frankie is sweeping, and Len is sorting locks on the counter. The CAT is watching from a safe distance, tail twitching.

FRANKIE
(teasing)
Ellie, you missed a spot.

Ellie looks down, mock-glaring at Frankie.

ELLIE
(defiant)
And you missed a broom.

[LAUGH TRACK]

Frankie chuckles, shaking his head.

FRANKIE
(smiling)
Just trying to keep you on your toes, kiddo.

Ellie rolls her eyes, but grins, continuing to dust. Suddenly, the ladder wobbles. Ellie gasps, trying to steady herself.

ELLIE
(scared)
Whoa!

Len looks up, alarmed. Frankie drops his broom, rushing to steady the ladder.

FRANKIE
(concerned)
Ellie!

Frankie manages to steady the ladder just in time. Ellie sighs in relief, smiling at Frankie.

ELLIE
(relieved)
Thanks, Frankie.

Frankie smiles, stepping back.

FRANKIE
(teasing)
Just part of the job.

[LAUGH TRACK]

Ellie rolls her eyes, but grins, c

## 🎬 Scene 19 Generation

In [ ]:
# Extract the description for Scene 19 from the episode outline
scene_19_desc = extract_scene(episode_outline, 19)
print(scene_19_desc)

Scene 19: "Unexpected Success" Frankie successfully helps a customer with a lock problem, resulting in an unexpected win and a confidence boost.


In [ ]:
# Generate the full script for Scene 19 using its description and the completed scripts for Scenes 16 to 18 as context
scene_19_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_19_desc,
    previous_scenes=[scene_16_script, scene_17_script, scene_18_script]
)

print(scene_19_script)

INT. EARL'S LOCKSMITH SHOP - LATE AFTERNOON

The shop is now clean and organized. The CAT is napping on a newly cleaned counter. Ellie is helping a CUSTOMER with a lock problem, while Frankie and Len watch from a distance.

CUSTOMER
(frantic)
I just can't seem to get this lock open!

ELLIE
(calm)
Don't worry, sir. We'll figure this out.

Frankie watches, looking worried.

FRANKIE
(whispering to Len)
Do you think she can handle it?

LEN
(smiling)
Let's see what she's got.

Ellie examines the lock, then tries a few keys. The lock doesn't open. She looks at the keys, then at the lock, deep in thought.

ELLIE
(thinking aloud)
Maybe... 

She tries a different key. The lock clicks open. The customer looks amazed.

CUSTOMER
(astonished)
You did it!

ELLIE
(smiling)
Just a little bit of detective work.

Frankie and Len share a surprised look.

FRANKIE
(amazed)
She did it.

LEN
(smiling)
Told you she had potential.

[LAUGH TRACK]

Frankie looks proud, walking over to Ellie.

FRANKIE
(proud)
Gre

## 🎬 Scene 20 Generation

In [ ]:
# Extract the description for Scene 20 from the episode outline
scene_20_desc = extract_scene(episode_outline, 20)
print(scene_20_desc)

Scene 20: "Locking Up" The trio end their first day running the locksmith shop together, closing the shop with a feeling of accomplishment and anticipation for what's next.


In [ ]:
# Generate the full script for Scene 20 using its description and the completed scripts for Scenes 17 to 19 as context
scene_20_script = generate_scene_baseline(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_20_desc,
    previous_scenes=[scene_17_script, scene_18_script, scene_19_script]
)

print(scene_20_script)

INT. EARL'S LOCKSMITH SHOP - EVENING

The shop is now spotless. Locks and keys are organized neatly on the shelves. Ellie, Frankie, and Len are standing at the counter, finishing up their work. The CAT is napping peacefully on the counter.

FRANKIE
(satisfied)
Well, we did it. The shop's never looked better.

LEN
(smiling)
And we didn't even break anything. That's a first.

[LAUGH TRACK]

Ellie grins, looking at the cleaned shop.

ELLIE
(teasing)
Well, not for lack of trying.

Frankie chuckles, shaking his head.

FRANKIE
(teasing)
Hey, I only knocked over that one shelf.

Ellie laughs, shaking her head.

ELLIE
(teasing)
Once an hour, maybe.

[LAUGH TRACK]

They all share a laugh, then go back to their work. Frankie looks at the clock, then at Ellie and Len.

FRANKIE
(smiling)
Alright, I think it's time to call it a day. What do you say?

Ellie and Len look at each other, then at Frankie.

ELLIE
(smiling)
I think that sounds like a great idea.

Len nods, smiling.

LEN
(smiling)
I agree.

## Combining Scripts

In [ ]:
# List your scene scripts in order
scene_scripts = [
    scene_1_script,
    scene_2_script,
    scene_3_script,
    scene_4_script,
    scene_5_script,
    scene_6_script,
    scene_7_script,
    scene_8_script,
    scene_9_script,
    scene_10_script,
    scene_11_script,
    scene_12_script,
    scene_13_script,
    scene_14_script,
    scene_15_script,
    scene_16_script,
    scene_17_script,
    scene_18_script,
    scene_19_script,
    scene_20_script
]

# Combine with optional scene headers
full_episode_script = "\n\n".join([
    f"### Scene {i+1} ###\n{script.strip()}"
    for i, script in enumerate(scene_scripts)
])

# Preview the first 500 characters
print(full_episode_script[:500])

### Scene 1 ###
# Scene 1

INT. PRISON GATES - DAY

FRANKIE, early 50s, a bit worn but with a twinkle in his eye, steps through the prison gates, suitcase in hand.

FRANKIE
(squinting against the sunlight)
Well, Frankie, welcome to the rest of your life.

[LAUGH TRACK]

A beat-up taxi pulls up. AL, the taxi driver, 60s, gruff but friendly, leans out the window.

AL
You Frankie?

FRANKIE
(smiles)
Depends. You the Feds?

[LAUGH TRACK]

AL
(chuckles)
Nope, just your chariot to freedom.

FRANKIE
(si


In [ ]:
path = "/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/pilot episode data/pilot episode 5/baseline_pilot_ep_scrpt_5.txt"

with open(path, "w") as f:
    f.write(full_episode_script)

print(f"Script saved to {path}")

Script saved to /content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/pilot episode data/pilot episode 5/baseline_pilot_ep_scrpt_5.txt
